In [2]:
#!/usr/bin/python
import os, sys, glob, icecube, healpy
import numpy as np
import matplotlib
%matplotlib inline
import matplotlib.colors as colors
from matplotlib import pyplot
from icecube import recclasses, dataclasses, dataio, icetray, millipede, gulliver
from icecube.recclasses import *
from icecube.astro import I3GetEquatorialFromDirection

import scipy
from scipy import misc, sparse
from scipy.stats import chi2, norm

In [65]:
filelist = glob.glob("../numu_pegleg/*")
filelist.sort()
# Limit this to 20 files by sorting by original
# file number
filenums = {int(f.split(".")[-4]) for f in filelist}
by_filenum = []
print filelist[0]
for n in filenums:
    by_filenum.append([f for f in filelist 
                       if '{:06d}.000'.format(n) in f])
    
filelist = by_filenum[0][:5]
print len(filelist)
print filelist

nside = 32
npix = healpy.nside2npix(nside)
hp_xyz = np.array(healpy.pix2vec(nside, np.arange(npix)))
hpdec, hpra = healpy.pix2ang(nside, np.arange(npix))
zenith, azimuth = np.pi-np.copy(hpdec), np.copy(hpra)
hpdec = np.pi/2 - hpdec

points = np.zeros((len(zenith), 2), dtype=float)
iz, ia = 0, 1
points[:,iz] = 0.5*(np.cos(zenith))+0.5
points[:,ia] = azimuth / (2*np.pi)

../numu_pegleg/Nestle_NuMu.140000.000001.000000.i3.bz2
5
['../numu_pegleg/Nestle_NuMu.140000.000001.000000.i3.bz2', '../numu_pegleg/Nestle_NuMu.140000.000001.000001.i3.bz2', '../numu_pegleg/Nestle_NuMu.140000.000001.000002.i3.bz2', '../numu_pegleg/Nestle_NuMu.140000.000001.000003.i3.bz2', '../numu_pegleg/Nestle_NuMu.140000.000001.000004.i3.bz2']


In [66]:
# Need to handle stuff outside of the unit cube...
# zenith reflects back while azimuth changes by 1
def handle_wrapping(endpoints):
    is_outside = (endpoints[:,iz] < 0) 
    if np.any(is_outside):
        endpoints[:,iz][is_outside] *= -1
        endpoints[:,ia][is_outside] += 0.5
    is_outside = (endpoints[:,iz] >= 1) 
    if np.any(is_outside):
        endpoints[:,iz][is_outside] *= -1
        endpoints[:,iz][is_outside] += 2
        endpoints[:,ia][is_outside] += 0.5
    endpoints[:,ia] %= 1

def to_xyz(az, zen):
    return np.array([np.cos(az)*np.sin(zen),
                     np.sin(az)*np.sin(zen),
                     np.cos(zen)])

# Define a logsinh function for numerical stability
def logsinh(x):
    return x - np.log(2) + np.log(1-np.exp(-2*x))
    return y
        

In [67]:
def get_map(ellipsoids, bf_llh=0, ia=1, iz=0):
    kent_sum = np.ones(points.shape[0],dtype=float) * np.inf
    for i in range(len(ellipsoids)):
        ell = ellipsoids[i]
        center = ell.center

        dllh = (ell.nllh - bf_llh)

        # Okay. Now what?
        l, v = np.linalg.eigh(ell.inverse_covariance)
        axlens = 1.0/np.sqrt(l)
        axes = np.dot(v, np.diag(axlens))

        major = axes[:, np.argmax(axlens)]
        minor = axes[:, np.argmin(axlens)]

        endpoints = np.array([center-major, center-minor])
        #endpoints2 = np.array([center+major, center+minor])

        handle_wrapping(endpoints)
        #handle_wrapping(endpoints2)

        # Need to convert the axes coordinates to physical coordinates...
        center[ia] *= 2*np.pi
        center[iz] = np.pi-np.arccos(2*center[iz]-1)

        endpoints[:,ia] *= 2*np.pi
        endpoints[:,iz] = np.pi-np.arccos(2*endpoints[:,iz]-1)

        #endpoints2[:,ia] *= 2*np.pi
        #endpoints2[:,iz] = np.pi-np.arccos(2*endpoints2[:,iz]-1)

        center = to_xyz(center[ia], center[iz])
        endpoints = np.array([to_xyz(endpoints[0,ia], endpoints[0,iz]),
                              to_xyz(endpoints[1,ia], endpoints[1,iz])])
        #endpoints2 = np.array([to_xyz(endpoints2[0,ia], endpoints2[0,iz]),
        #                      to_xyz(endpoints2[1,ia], endpoints2[1,iz])])

        #axis2 = (endpoints[0] - endpoints2[0])/2.
        #axis3 = (endpoints[1] - endpoints2[1])/2.
        axis2 = endpoints[0] - center
        axis3 = endpoints[1] - center
        
        gamma2_length = np.dot(axis2,axis2)**0.5
        gamma3_length = np.dot(axis3,axis3)**0.5
        axlens = np.array([gamma2_length, gamma3_length])
        if np.any(np.isnan(axlens)): continue    

        # I want gamma3 to be the minor axis. Swap them
        # if this isn't the case
        if gamma2_length < gamma3_length:
            tmp = np.copy(axis2)
            axis2 = np.copy(axis3)
            axis3 = tmp

        # We have the kappa value. Let's get the gamma1/2/3
        gamma1 = center / np.sqrt(np.sum(center**2))
        gamma2 = axis2 / np.sqrt(np.sum(axis2**2))
        gamma3 = axis3 / np.sqrt(np.sum(axis3**2))

        # The axes were orthogonal in azimuth/coszen, not RA/dec
        # The major/minor axes are already orthogonal to the central
        # axis, so we just need to project one of them into a third
        # axis in order to have "good" representation. I'm going to
        # just project the minor axis.
        g3 = np.cross(gamma1, gamma2)
        g3_length = (np.dot(axis3, g3)/(np.dot(g3,g3)**0.5))

        axis3 = g3 * g3_length
        gamma3 = g3

        sigma = np.mean(axlens)
        kappa = 1./(sigma)**2
        beta = 0.5*kappa * np.sqrt(1-axlens.min()**2/axlens.max()**2)
        
        # How should we modify the loglikelihood 
        llh_weight = dllh #+ 2*np.log(1-ell.correlation)

        # We now have to calculate the actual llh at each point
        # Going to try truncating the constant to just c(0,0)
        # Normalization constant:
        current_llh = -np.log(kappa)
        current_llh += np.log(4*np.pi)
        current_llh += logsinh(kappa)

        # Shape parameters
        current_llh -= kappa*np.dot(gamma1, hp_xyz)
        current_llh -= beta*np.dot(gamma2, hp_xyz)**2
        current_llh += beta*np.dot(gamma3, hp_xyz)**2
        kent_sum = -misc.logsumexp([-kent_sum,
                                    -current_llh-llh_weight],axis=0)

    return kent_sum

In [ ]:
# Build the structures to hold the information I want
run = []
event = []
subevent = []
time = []

reco_energy = []
reco_zenith = []
reco_azimuth = []
reco_ra = []
reco_dec = []

llh_map = []

# MC Only
is_mc = False
oneweight = []
true_energy = []
true_ra = []
true_dec = []
dllh_truth = []

for filenum, infile in enumerate(filelist):
    i3file = dataio.I3File(infile, 'r')        
    print filenum,'of',len(filelist), ':', infile
    eventnum = 0
    while i3file.more():
        try: frame = i3file.pop_physics()
        except: 
            print("Something broke")
            break
        
        eventnum += 1
        
        # Event header information
        header = frame['I3EventHeader']
        run.append(header.run_id)
        event.append(header.event_id)
        subevent.append(header.sub_event_id)
        
        t = header.start_time
        time.append(t.mod_julian_day_double)

        if 'I3MCTree' in frame.keys():
            is_mc = True

            # Get the information
            truth = dataclasses.get_most_energetic_neutrino(frame['I3MCTree'])
            mcwd = frame['I3MCWeightDict']
            
            # Scale by the number of events
            ow = mcwd['OneWeight']/mcwd['NEvents']
            if 'genie' in infile:
                if truth.pdg_encoding > 0: 
                    ow/=0.7
                else: 
                    ow/=0.3
            else:
                ow /= 0.5            

            # Intentionally not going to scale by
            # nfiles here, since I want to do it when
            # I merge files. That'll make my life easier
            # when I add more files to these.
            oneweight.append(ow)
            
            x = I3GetEquatorialFromDirection(truth.dir, t)
            true_ra.append(x.ra)
            true_dec.append(x.dec)
            true_energy.append(truth.energy)
        
        # Reconstruction values    
        bestfit = frame['Pegleg_Fit_NestleTrack']
        
        reco_energy.append(bestfit.energy)
        reco_zenith.append(bestfit.dir.zenith)
        reco_azimuth.append(bestfit.dir.azimuth)

        x = I3GetEquatorialFromDirection(bestfit.dir, t)
        reco_ra.append(x.ra)
        reco_dec.append(x.dec)
        
        bf_llh = frame['Pegleg_Fit_NestleFitParams'].logl
        ellipses = frame['Pegleg_Fit_Nestle_NestleMinimizer']

        # Convert the truth into something usable with the ellipsoids
        ellipses = ellipses.prune(200)
        names = list(ellipses.axis_names)

        iz, ia = names.index('Zenith'), names.index('Azimuth')
        remove = np.ones(len(names), dtype=bool)
        remove[iz] = False
        remove[ia] = False
        ellipses = ellipses.profile(np.arange(len(names))[remove])
        names = list(ellipses.axis_names)
        iz, ia = names.index('Zenith'), names.index('Azimuth')
        ellipses = [ellipses[i] for i in range(len(ellipses))]
        
        # Make a map for this event?
        kent_sum = get_map(ellipses, bf_llh, ia, iz)
        llh_map.append(kent_sum)
        
        # Where is the truth in this map?
        if is_mc:
            truth_bin = healpy.ang2pix(nside, np.pi-truth.dir.zenith, truth.dir.azimuth)
            dllh = np.max(kent_sum) - kent_sum[truth_bin]
            dllh_truth.append(dllh)
        
        del ellipses        

0 of 5 : ../numu_pegleg/Nestle_NuMu.140000.000001.000000.i3.bz2
1 of 5 : ../numu_pegleg/Nestle_NuMu.140000.000001.000001.i3.bz2
2 of 5 : ../numu_pegleg/Nestle_NuMu.140000.000001.000002.i3.bz2
3 of 5 : ../numu_pegleg/Nestle_NuMu.140000.000001.000003.i3.bz2
4 of 5 : ../numu_pegleg/Nestle_NuMu.140000.000001.000004.i3.bz2


In [ ]:
run      = np.array(run, dtype=int)
event    = np.array(event, dtype=int)
subevent = np.array(subevent, dtype=int)
time     = np.array(time, dtype=float)

reco_energy  = np.array(reco_energy, dtype=float)
reco_zenith  = np.array(reco_zenith, dtype=float)
reco_azimuth = np.array(reco_azimuth, dtype=float)
reco_ra  = np.array(reco_ra, dtype=float)
reco_dec = np.array(reco_dec, dtype=float)

llh_map = np.array(llh_map, dtype=float)

if is_mc:
    oneweight    = np.array(oneweight, dtype=float)
    true_energy  = np.array(true_energy, dtype=float)
    true_ra      = np.array(true_ra, dtype=float)
    true_dec     = np.array(true_dec, dtype=float)
    dllh_truth   = np.array(dllh_truth, dtype=float)


In [ ]:
# Write it out?
if is_mc:
    datatypes = np.dtype( [ 
        ('run', np.int64), ('event', np.int64),
        ('subevent', np.int64), ('time', np.float64),
        ('ra', np.float64), ('dec', np.float64),
        ('azi', np.float64), ('zen', np.float64),
        ('angErr', np.float64), ('logE', np.float64),
        ('trueE', np.float64), ('trueRa', np.float64),
        ('trueDec', np.float64), ('ow', np.float64), ],
        ('trueDeltaLLH', np.float64))
    data = np.array([run, event, subevent, time,
                    reco_ra, reco_dec, reco_azimuth, reco_zenith,
                    np.zeros_like(reco_zenith), 
                    np.log10(reco_energy), true_energy,
                    true_ra, true_dec, oneweight,
                    dllh_truth], datatypes).T
else:
    datatypes = np.dtype( [ 
        ('run', np.int64), ('event', np.int64),
        ('subevent', np.int64), ('time', np.float64),
        ('ra', np.float64), ('dec', np.float64),
        ('azi', np.float64), ('zen', np.float64),
        ('angErr', np.float64), ('logE', np.float64) ] )
    
    data = np.array([run, event, subevent, time,
                    reco_ra, reco_dec, reco_azimuth, reco_zenith,
                    np.zeros_like(reco_zenith), 
                    np.log10(reco_energy)], datatypes).T
    
output_name = 'test'
print data.dtype.names
data = np.rec.array(data, dtype=datatypes)

np.save(output_name, data)
np.save(output_name+"_maps", llh_map)
